In [1]:
import random
import string

random.seed(0)
MAX_DEPTH = 5

def random_label(num=1, chars=string.ascii_lowercase):
    return ''.join(random.choices(chars, k=num))

def random_permutation(k):
    return random.sample(range(k), k)

def random_names(k,num=1):
    return [random_label(num) for _ in range(k)]

MORPHS = ['morphism', 'morphism_with_names']
CROSS = ['crossing', 'crossing_with_names']
COMPTENS = ['compose', 'tensor']

def random_object(no_crossing=False,no_comptens = False,prob=5):
    global MORPHS, CROSS, COMPTENS
    if no_comptens and no_crossing:
        return random.choices(MORPHS, weights=[1, 1], k=1)[0]
    if no_comptens:
        return random.choices(MORPHS+CROSS, weights=[1, 1, 1, 1], k=1)[0]
    if no_crossing:
        return random.choices(MORPHS+COMPTENS, weights=[1, 1, prob, prob*1.5], k=1)[0]
    return random.choices(MORPHS+CROSS+COMPTENS
                , weights=[1, 1, 1, 1, prob, prob*1.5], k=1)[0]

def morphism(l,r):
    return {"type": "Morphism", "label": random_label(), "arity": [l,r]}

def morphism_with_names(l,r):
    return {"type": "MorphismWNames", "label": random_label(), "arityL": random_names(l), "arityR": random_names(r)}

def crossing(k):
    return {"type": "Crossing", "permutation": random_permutation(k)}

def crossing_with_names(k):
    return {"type": "CrossingWNames", "permutation": random_permutation(k), "arity": random_names(k)}

def compose(l,r,d):
    global MAX_DEPTH, MORPHS, CROSS
    k = random.randint(1, (l+r)//2 + 1)
    f1 = random_object() if d < MAX_DEPTH else random_object(no_comptens=True)
    diagram1 = eval(f1)(l) if f1 in CROSS \
        else eval(f1)(l,k) if f1 in MORPHS \
        else eval(f1)(l,k,d+1)
    if f1 in CROSS: k = l
    f2 = random_object(no_crossing=True) if d < MAX_DEPTH else random_object(no_crossing=True,no_comptens=True)
    diagram2 = eval(f2)(k,r) if f2 in MORPHS \
        else eval(f2)(k,r,d+1)
    return {"type": "Compose", "diagram1": diagram1, "diagram2": diagram2}

def tensor(l,r,d):
    global MAX_DEPTH, MORPHS
    if l == 1 or r == 1: return compose(l,r,d)
    l1 = random.randint(1, l-1)
    l2 = l - l1
    r1 = random.randint(1, r-1)
    r2 = r - r1
    f1 = random_object(no_crossing=True) if d < MAX_DEPTH else random_object(no_crossing=True,no_comptens=True)
    diagram1 = eval(f1)(l1,r1) if f1 in MORPHS \
        else eval(f1)(l1,r1,d+1)
    f2 = random_object(no_crossing=True) if d < MAX_DEPTH else random_object(no_crossing=True,no_comptens=True)
    diagram2 = eval(f2)(l2,r2) if f2 in MORPHS \
        else eval(f2)(l2,r2,d+1)
    return {"type": "Tensor", "diagram1": diagram1, "diagram2": diagram2}

def generate_random_object():
    global MORPHS, CROSS
    l = random.randint(5, 10)
    r = random.randint(5, 10)
    f = random_object(prob=30, no_crossing=True)
    return eval(f)(l) if f in CROSS \
         else eval(f)(l,r) if f in MORPHS \
         else eval(f)(l,r,1)

In [2]:
import json

for i in range(100):
    with open(f"gen/test{i}.json", "w") as f:
        random_json_object = generate_random_object()
        json.dump(random_json_object, f, indent=4)